# BitArray

Um bequem mit den einzelnen Bits zu operieren wird eine Klasse `BitArray` erstellt, welche den wert als `int` abspeichert und Methoden zur Auslesung, Überschreibung, Sequenzierung und Konkatenation ermöglicht.

In [1]:
class BitArray:
    def __init__(self, bits=None, length=None):
        if bits is None:
            self.array = 0
            self.length = 0
        elif type(bits) in {chr, str}:
            self.array = int(bits, 2)
            self.length = len(bits)
        elif type(bits) == int:
            self.array = bits
            self.length = length if length else bits.bit_length()
        elif type(bits) == BitArray:
            self.array = bits.array
            self.length = bits.length
        else:
            raise Exception(f'Bits must be of type str or int. {type(bits)} is something completly different.')
    
    def __call__(self, step=1):
        for i in range(0, len(self), step):
            if i+step <= len(self):
                yield self[i:i+step]
    
    def __getitem__(self, idx):
        if type(idx) == slice:
            start = (idx.start if idx.start is not None else 0) % len(self)
            stop = (idx.stop if idx.stop is not None else len(self)) % (len(self) + 1)
            return BitArray((self.array >> len(self) - stop) & ((1 << stop - start) - 1), length=stop - start)
        else:
            return BitArray(self.array >> idx % len(self) & 1, length=1)
    
    def __setitem__(self, idx, value):
        if type(value) != BitArray:
            raise Exception(f'Value must be of type BitArray. {type(value)} is something completly different.')
        cutoff = min(len(self), idx + len(value))
        self.array = (self[:idx]|value[:cutoff]|self[cutoff:]).array
    
    def __or__(self, other):
        return BitArray(self.array << len(other)|other.array, length=len(self) + len(other))
    
    def __repr__(self):
        return f'BitArray[{str(self)}]'
    
    def __str__(self):
        if self.array:
            return bin(self.array)[2:].zfill(len(self))[:len(self)]
        else:
            return ''
    
    def __len__(self):
        return self.length
    
    def __int__(self):
        return self.array
    
    def __eq__(self, other):
        return self.array == other.array and len(self) == len(other)

# Morse Code

Die Morsekodierung bildet von der Menge des Alphabets auf eine Menge bestehend aus weniger Zeichen ab. Die Kodierung für ein einzelnes Zeichen ist damit:

$$ \Omega := Alphabet \cup Numerals \cup  \{ {., \_} \}$$
$$ \Sigma :=  \{ .  \_ : *   \} $$
$$ \mathcal{M}: \Omega \mapsto Morse \: Code \subset \Sigma ^{+}$$

Um die Kodierung mehrer Zeichen $Morse^n$ zu ermöglichen, muss eine Trennung der einzelnen Zeichen aus $Morse$ erfolgen, um die Eindeutligkeit zu gewähren. Dies wird auf der Ebene des Morsecodes durch eine Trennung per Leerzeichen `SPACE` zur Abgrenzung der einzelnen Alphabetsbuchstaben ermöglicht:

$$ \mathcal{M}^1: \Omega \mapsto \mathcal{M} $$ 
$$ \mathcal{M}^{n\neq 1}: \Omega^n \mapsto \mathcal{M} \circ \_ \circ \mathcal{M}^{n-1} $$

Die dadurch erzeugte Folge kann nun weiter binär kodiert werden. Hierzu werden die Elemente aus $\Sigma$ auf 2 Bit abgebildet.

$$ \mathcal{B} : \Sigma \mapsto \{0, 1\}^2$$

Da in $\mathcal{M}^n$ jedoch noch das Zeichen `SPACE` enthalen ist, muss eine binäre Kodierung für $\mathcal{M}^n$ die Abbildung $\mathcal{B}$ erweitern. Eine naive Methode wäre es, $\Sigma$ um $\_$ zu erweitern, dies hätte jedoch den nachteil, dass wir dann 3 Bit für die Abbildung $\mathcal{B}$ bräuchten. Da $\forall c \in \Omega: |\mathcal{M}(c)| \in [1, 5]$, können wir für jedes `SPACE` $\in \mathcal{M}^n$ die Schritweite $\Lambda$ bis zum nächten $\_$ kodieren.

$$\Lambda: \mathcal{M} \mapsto \{0, 1\}^2$$
$$\Lambda(m) := 2\cdot(|m|-1)_{bin}$$

Damit kann beim Dekodieren $\mathcal{\bar{B}}^n$ zwischen den Schritten des Auslesens der Länge des nächsten Blocks $\Lambda(m)$ und dem Auslesen des nächsten Blocks $\mathcal{\bar{B}}$ alterniert werden.

$$\mathcal{B}^m(x) \mapsto \mathcal{B}^n\circ\mathcal{M}(x_0)|\Lambda\circ\mathcal{B}^n\circ\mathcal{M}(x_0) \dots \mathcal{B}^n\circ\mathcal{M}(x_{m/n})|\Lambda\circ\mathcal{B}^n\circ\mathcal{M}(x_{m/n})$$

### Morsecode

<!--<blockquote> -->
    
$\mathcal{M}^5($ `WITCH.` $) \mapsto \mathcal{M}($ `W` $)$ `SPACE` $\mathcal{M}($ `I` $)$ `SPACE` $\mathcal{M}($ `T` $)$ `SPACE`$\mathcal{M}($ `C` $)$ `SPACE` $\mathcal{M}($ `H` $) \mathcal{M}($ `.` $)$

$\mathcal{M}($ `W` $) \mapsto $ `.__`

$\mathcal{M}($ `I` $) \mapsto $ `..`

$\mathcal{M}($ `T` $) \mapsto $ `_`

$\mathcal{M}($ `C` $) \mapsto $ `_._.`

$\mathcal{M}($ `H` $) \mapsto $ `....`

$\mathcal{M}($ `.` $) \mapsto $ `*`

$\mathcal{M}^5($ `WITCH.` $) \mapsto$ `.__ .. _ _._. .... *`

### Binärcode

$\mathcal{B}^m($ `WITCH.` $) \mapsto \mathcal{B}^n($ `.__` $)| \Lambda\circ\mathcal{B}^n($ `.__`)| $\mathcal{B}^n($ `..` $)| \Lambda\circ\mathcal{B}^n($ `..`)| $\mathcal{B}^n($ `_` $)| \Lambda\circ\mathcal{B}^n($ `_`)| $\mathcal{B}^n($ `_._.` $)| \Lambda\circ\mathcal{B}^n($ `_._.` )| $\mathcal{B}^n($ `....` $)| \Lambda\circ\mathcal{B}^n($ `....` $)| \mathcal{B}^n($ `*` $)| \Lambda\circ\mathcal{B}^n($ `*` $)$

$\mathcal{B}^n($ `.__` $) \mapsto \mathcal{B}($ `.` $) \mathcal{B}($ `_` $) \mathcal{B}($ `_` $) = $ `000101`

$\Lambda($ `000101` $) \mapsto $ `10`

$\mathcal{B}^n($ `..` $)\mapsto\mathcal{B}($ `.` $) \mathcal{B}($ `.` $) = $ `0000`

$\Lambda($ `0000` $) \mapsto $ `01`

$\mathcal{B}^n($ `_` $)\mapsto\mathcal{B}($ `_` $) = $ `01`

$\Lambda($ `01` $) \mapsto $ `00`

$\mathcal{B}^n($ `_._.` $)\mapsto\mathcal{B}($ `_` $) \mathcal{B}($ `.` $) \mathcal{B}($ `_` $) \mathcal{B}($ `.` $) = $ `01000100`

$\Lambda($ `01000100` $) \mapsto $ `11`

$\mathcal{B}^n($ `....` $)\mapsto\mathcal{B}($ `.` $) \mathcal{B}($ `.` $) \mathcal{B}($ `.` $) \mathcal{B}($ `.` $) = $ `00000000`

$\Lambda($ `00000000` $) \mapsto $ `11`

$\mathcal{B}^n($ `*` $)\mapsto\mathcal{B}($ `*` $) = $ `11`

$\Lambda($ `11` $) \mapsto $ `00`

$\mathcal{B}^m($ `WITCH.` $) \mapsto $ `100001010100000001110100010011000000000011`

Das Ergebnis wird dann gelesen als:

(`10` 3 `000101` W), (`01` 2 `0000` I), (`00` 1 `01` T), (`11` 4 `01000100` C), (`11` 4 `00000000` H), (`00` 1 `11`.)


In [2]:
class Encoder:
    def __init__(self):
        self.morse = {'A': '._', 'B': '_...', 'C': '_._.', 'D': '_..', 'E': '.', 'F': '.._.', 'G': '__.', 'H': '....', 'I': '..', 'J': '.___', 'K': '_._', 'L': '._..', 'M': '__', 'N': '_.', 'O': '___', 'P': '.__.', 'Q': '__._', 'R': '._.', 'S': '...', 'T': '_', 'U': '.._', 'V': '..._', 'W': '.__', 'X': '_.._', 'Y': '_.__', 'Z': '__..', '0': '_____', '1': '.____', '2': '..___', '3': '...__', '4': '...._', '5': '.....', '6': '_....', '7': '__...', '8': '___..', '9': '____.', ' ': ':', '.': '*',}
        self.morse_inv = {v: k for k, v in self.morse.items()}
        self.binary = {'.': 0b00, '_': 0b01, ':': 0b10, '*': 0b11,}
        self.binary_inv = {v: k for k, v in self.binary.items()}
    
    def encode(self, x, status=False):
        if set(self.binary.keys()) - set(x):
            x = self.encode_morse(x)
            if status: print(x)
        x = self.encode_binary(x)
        return x

    def decode(self, x, status=False):
        if isinstance(x, BitArray):
            x = self.decode_binary(x)
            if status: print(x)
        x = self.decode_morse(x)
        return x
    
    def encode_morse(self, text):
        return ' '.join(self.morse[char] for char in text.upper() if char in self.morse)

    def decode_morse(self, text):
        return ''.join(self.morse_inv[code] for code in text.split(' ') if code in self.morse_inv)
    
    def encode_binary(self, codes):
        stream = BitArray()
        for code in codes.split(' '):
            stream = stream|BitArray(len(code)-1, 2)
            for char in code:
                stream = stream|BitArray(self.binary[char], length=2)
        return stream
    

    def decode_binary(self, stream):
        chars, i = [], 0
        while i < len(stream):
            char_length = (int(stream[i:i+2])+1)*2
            chars.append(''.join(self.binary_inv[int(x)] for x in stream[i+2:i+2+char_length](2)))
            i += 2 + char_length 
        return ' '.join(chars)
    
    def read(self, file, status=False):
        with open(file, 'r') as file:
            x = file.read().strip()
            if set(x) - {'0', '1'}:
                return self.encode(x, status)
            else:
                return self.decode(BitArray(x), status)
    
    def write(self, x, file, status=False):
        with open(file, 'w') as file:
            if isinstance(x, BitArray):
                file.write(self.decode(x, status))
            else:
                file.write(str(self.encode(x, status)))

In [3]:
witch = """
Tell me, what do you do with witches. 
Burn Burn Burn... 
What do you burn apart from witches. 
More witches... 
Wood. 
So, why do witches burn. 
Cause they're made of... wood. 
So, how do we tell if she is made of wood. 
Build a bridge out of her. 
Ahh, but can you not also make bridges out of stone. 
Oh yeah... 
Does wood sink in water. 
No. It floats. 
Let's throw her into the bog. 
What also floats in water. 
Bread. 
Apples. 
Very small rocks. 
Cider. 
Grape gravy. 
Cherries. 
Mud. 
A Duck. 
Exactly. So, logically... 
If she ways the same as a duck... she's made of wood. 
And therefore... A witch. 
We shall use my largest scales. 
"""

enc = Encoder()
stream = enc.encode(witch, True)
print(repr(stream))

_ . ._.. ._.. : __ . : .__ .... ._ _ : _.. ___ : _.__ ___ .._ : _.. ___ : .__ .. _ .... : .__ .. _ _._. .... . ... * : _... .._ ._. _. : _... .._ ._. _. : _... .._ ._. _. * * * : .__ .... ._ _ : _.. ___ : _.__ ___ .._ : _... .._ ._. _. : ._ .__. ._ ._. _ : .._. ._. ___ __ : .__ .. _ _._. .... . ... * : __ ___ ._. . : .__ .. _ _._. .... . ... * * * : .__ ___ ___ _.. * : ... ___ : .__ .... _.__ : _.. ___ : .__ .. _ _._. .... . ... : _... .._ ._. _. * : _._. ._ .._ ... . : _ .... . _.__ ._. . : __ ._ _.. . : ___ .._. * * * : .__ ___ ___ _.. * : ... ___ : .... ___ .__ : _.. ___ : .__ . : _ . ._.. ._.. : .. .._. : ... .... . : .. ... : __ ._ _.. . : ___ .._. : .__ ___ ___ _.. * : _... .._ .. ._.. _.. : ._ : _... ._. .. _.. __. . : ___ .._ _ : ___ .._. : .... . ._. * : ._ .... .... : _... .._ _ : _._. ._ _. : _.__ ___ .._ : _. ___ _ : ._ ._.. ... ___ : __ ._ _._ . : _... ._. .. _.. __. . ... : ___ .._ _ : ___ .._. : ... _ ___ _. . * : ___ .... : _.__ . ._ .... * * * : _.. ___ . ... : .__ ___

In [4]:
text = enc.decode(stream, True)
print(text)

_ . ._.. ._.. : __ . : .__ .... ._ _ : _.. ___ : _.__ ___ .._ : _.. ___ : .__ .. _ .... : .__ .. _ _._. .... . ... * : _... .._ ._. _. : _... .._ ._. _. : _... .._ ._. _. * * * : .__ .... ._ _ : _.. ___ : _.__ ___ .._ : _... .._ ._. _. : ._ .__. ._ ._. _ : .._. ._. ___ __ : .__ .. _ _._. .... . ... * : __ ___ ._. . : .__ .. _ _._. .... . ... * * * : .__ ___ ___ _.. * : ... ___ : .__ .... _.__ : _.. ___ : .__ .. _ _._. .... . ... : _... .._ ._. _. * : _._. ._ .._ ... . : _ .... . _.__ ._. . : __ ._ _.. . : ___ .._. * * * : .__ ___ ___ _.. * : ... ___ : .... ___ .__ : _.. ___ : .__ . : _ . ._.. ._.. : .. .._. : ... .... . : .. ... : __ ._ _.. . : ___ .._. : .__ ___ ___ _.. * : _... .._ .. ._.. _.. : ._ : _... ._. .. _.. __. . : ___ .._ _ : ___ .._. : .... . ._. * : ._ .... .... : _... .._ _ : _._. ._ _. : _.__ ___ .._ : _. ___ _ : ._ ._.. ... ___ : __ ._ _._ . : _... ._. .. _.. __. . ... : ___ .._ _ : ___ .._. : ... _ ___ _. . * : ___ .... : _.__ . ._ .... * * * : _.. ___ . ... : .__ ___

In [5]:
file_name = 'witch.b'
enc.write(witch, file_name)
print(enc.read(file_name))

TELL ME WHAT DO YOU DO WITH WITCHES. BURN BURN BURN... WHAT DO YOU BURN APART FROM WITCHES. MORE WITCHES... WOOD. SO WHY DO WITCHES BURN. CAUSE THEYRE MADE OF... WOOD. SO HOW DO WE TELL IF SHE IS MADE OF WOOD. BUILD A BRIDGE OUT OF HER. AHH BUT CAN YOU NOT ALSO MAKE BRIDGES OUT OF STONE. OH YEAH... DOES WOOD SINK IN WATER. NO. IT FLOATS. LETS THROW HER INTO THE BOG. WHAT ALSO FLOATS IN WATER. BREAD. APPLES. VERY SMALL ROCKS. CIDER. GRAPE GRAVY. CHERRIES. MUD. A DUCK. EXACTLY. SO LOGICALLY... IF SHE WAYS THE SAME AS A DUCK... SHES MADE OF WOOD. AND THEREFORE... A WITCH. WE SHALL USE MY LARGEST SCALES. 


In [6]:
def morse_ascii(text):
    b = BitArray()
    for char in text:
        b = b|BitArray(ord(char), length=7)
    return b


ascii_encoding = morse_ascii(witch)

print(f"""
Length of Morse->ASCII encoding:\t{len(ascii_encoding)} bit
Length of Morse->Binary encoding:\t{len(stream)} bit
Redutcion by:\t{round(len(ascii_encoding)/len(stream)*100, 1)}%
""")


Length of Morse->ASCII encoding:	4508 bit
Length of Morse->Binary encoding:	3944 bit
Redutcion by:	114.3%

